# TP2
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Problema 1

### Descrição do Problema

É nos dado um Control Flow Automaton (CFA) que descreve um programa imperativo cujo objetivo é implementar a multiplicação de dois inteiros a,b, fornecidos como "input" e um n, também fornecido como "input", de precisão limitada.
Para alem disso, temos de ter em conta os seguintes aspetos:
- Existe a possibilidade de alguma das operações do programa produzir um erro de “overflow”;
- Os nós do grafo representam ações  que actuam sobre os “inputs” do nó e produzem um “output” com as operações indicadas;
- Os ramos do grafo representam ligações que transferem o “output” de um nodo para o “input” do nodo seguinte. Esta transferência é condicionada pela satisfação da condição associada ao ramo.

### Abordagem do Problema

Para resolver este problema, vamos construir um First Order Transition System (FOTS) usando um BitVector de tamanho n de forma a descrever o comportamento do autómato acima mencionado.

( Talvez seja fixe mencionar a escolha do solver aqui futuramente)

Para além do FOTS, também vamos verificar se $P≡(x∗y+z=a∗b)$ é um invariante do comportamento que estamos a estudar.


## Código Python

variaveis -> x,y,z e o program counter (nº de linhas de código)

x,y,z = a,b,0

ideia 1:
0: while(y!=0):

1: 	if even(y) then x,y,z = 2*x,y/2,z

2:	if odd(y)  then x,y,z = x,y-1,z+x

3: stop

ideia 2:

0: while(y!=0):

 	if even(y) then x,y,z = 2*x,y/2,z

	if odd(y)  then x,y,z = x,y-1,z+x

1: stop


outras ideias que podem ser importantes: na aula T do dia 4 vimos que o stor evita usar o diferente por causa de ter problemas com indicibilidade, devemos também fazer o mesmo, vou cometer o erro aqui em baixo de usar not equals mas talvez temos de trocar depois.não esquecer

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import random as rn

# vou declarar aqui um exemplo depois passamos para exemplos separados
n = 32 # [0,0] [0,1] [1,0] [1,1]
a = 24
b = 12
k = 2*n
def toInt(s):
    return s # return sum([int(x)*2**(len(s)-i-1) for i,x in (enumerate(s))])

print('x = ', BV(a,n).bv_str())
print('y = ', BV(b,n).bv_str())

x =  00000000000000000000000000011000
y =  00000000000000000000000000001100


 $$\mathsf{trans}(x,y,z,pc,x',y',z',pc')\;\equiv\;$$
 $$\left\{\begin{array}{lr} (y=0)\land (pc=0) \land (x'=x)\land (y'=y) \land (c'=c) \land (pc'=3) & \lor \\ (a > b)\land(c=0)\land (a'+b=a) \land (b'=b)\land (c'=0) & \lor \\ (a < b)\land (c=0) \land (b'+a = b)\land (a'=a)\land (c'=0) & \lor \\ (c=1)\land(c'=1)\land (a'=a)\land (b'=b) & \end{array}\right.$$


In [2]:
# Ideias à toa, não faz muito sentido o que está escrito aqui a baixo

def BVLast(n):
    return BV(2**(n-1),n)

def geraBV(valor,n):       
    return BV(valor,n)

def bv_sel(z,i):
    return BVZExt(BVExtract(z,start=i,end=i),n-1)

def bv_hamm(z,y=None):       
    if y != None:
        return bv_hamm(BVXor(z,y))
    return sum([bv_sel(z,i) for i in range(n)])

def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)
    state['x'] = Symbol('x'+str(i),types.BVType(n))
    state['y'] = Symbol('y'+str(i),types.BVType(n))
    state['z'] = Symbol('z'+str(i),types.BVType(n))
    return state

def init(state):
    tPc = Equals(state['pc'],Int(0)) # Program counter a zero
    tZ = Equals(state['z'],BVZero(n)) # Z a zero
    tX = Equals(state['x'], BV(a,n)) # x inicilizado com valor de a
    tY = Equals(state['y'], BV(b,n)) # y inicilizado com valor de b
    return And(tPc,tX,tY,tZ)

def tStop(curr,prox):
    tYZero = Equals(curr['y'],BVZero(n))
    tPcZero = Equals(curr['pc'],Int(0))
    tYGe = BVUGE(curr['y'],BVZero(n))
    tX = Equals(prox['x'],curr['x'])
    tY = Equals(prox['y'],curr['y'])
    tZ = Equals(prox['z'],curr['z'])
    tPc = Equals(prox['pc'],Int(1))
    return And(tYZero,tPcZero,tYGe,tX,tY,tZ,tPc)

def tEven(curr,prox):
    tPcZero = Equals(curr['pc'],Int(0)) #dam este e o stop sao ambiguos
    tYLast = Equals(BVAnd(curr['y'],BVOne(n)),BVZero(n))
    tXFirst = Equals(BVAnd(curr['x'],BV(2**(n-1),n)),BVZero(n))
    tX = Equals(prox['x'], BVLShl(curr['x'],BVOne(n)))
    tY = Equals(prox['y'], BVLShr(curr['y'],BVOne(n)))
    tZ = Equals(prox['z'],curr['z'])
    tPc = Equals(prox['pc'],Int(0))
    return And(tPcZero,tYLast,tXFirst,tX,tY,tZ,tPc)

def tError(curr,prox):
    tPcZero = Equals(curr['pc'],Int(0))
    tYLast = Equals(BVAnd(curr['y'],BVOne(n)),BVZero(n))
    tXFirst = Equals(BVAnd(curr['x'],BVLast(n)),BVLast(n))
    tX = Equals(prox['x'], curr['x'])
    tY = Equals(prox['y'],curr['y'])
    tZ = Equals(prox['z'],curr['z'])
    tPc = Equals(prox['pc'],Int(2))
    return And(tPcZero,tYLast,tXFirst,tX,tY,tZ,tPc)

def tEndError(curr,prox):
    tPcTwo = Equals(curr['pc'],Int(2))
    tX = Equals(prox['x'], curr['x'])
    tY = Equals(prox['y'],curr['y'])
    tZ = Equals(prox['z'],curr['z'])
    tPc = Equals(prox['pc'],Int(2))
    return And(tPcTwo,tX,tY,tZ,tPc)

def tOdd(curr,prox):
    tPcZero = Equals(curr['pc'],Int(0))
    tYLast = Equals(BVAnd(curr['y'],BVOne(n)),BVOne(n))
    tX = Equals(prox['x'], curr['x'])
    tY = Equals(prox['y'],BVSub(curr['y'],BVOne(n)))
    tZ = Equals(prox['z'],BVAdd(curr['x'],curr['z'])) 
    tPc = Equals(prox['pc'],Int(0))   
    return And(tPcZero,tYLast,tX,tY,tZ,tPc)

def tEnd(curr,prox):
    tPcOne = Equals(curr['pc'],Int(1))
    tX = Equals(prox['x'],curr['x'])
    tY = Equals(prox['y'],curr['y'])
    tZ = Equals(prox['z'],curr['z'])
    tPc = Equals(prox['pc'],Int(1))
    return And(tPcOne,tX,tY,tZ,tPc)

def trans(curr,prox):
    tToStop = tStop(curr,prox)
    tToEven   = tEven(curr,prox)
    tToError  = tError(curr,prox)
    tToEndError = tEndError(curr,prox)
    tToOdd    = tOdd(curr,prox)
    tToEnd    = tEnd(curr,prox)
    return Or(tToStop,tToEven,tToError,tToEndError,tToOdd,tToEnd)



In [7]:
with Solver(name="z3") as s:
    
        # completar
    
        # cria k copias do estado
        trace = [declare(i) for i in range(k)]
        #print(trace)
        # criar o traço
        s.add_assertion(init(trace[0]))
        #print(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i], trace[i+1]))
        
        if s.solve():
            for i in range(k):
                print(i)
                print("pc=", pc := s.get_value(trace[i]['pc']).constant_value())
                print("x=", toInt(s.get_value(trace[i]['x']).bv_str()))
                print("y=", toInt(s.get_value(trace[i]['y']).bv_str()))
                print("z=", toInt(s.get_value(trace[i]['z']).bv_str()))
                print()
                if pc in (1,2):
                    break
        else:
            print('no')

0
pc= 0
x= 00000000000000000000000000011000
y= 00000000000000000000000000001100
z= 00000000000000000000000000000000

1
pc= 0
x= 00000000000000000000000000110000
y= 00000000000000000000000000000110
z= 00000000000000000000000000000000

2
pc= 0
x= 00000000000000000000000001100000
y= 00000000000000000000000000000011
z= 00000000000000000000000000000000

3
pc= 0
x= 00000000000000000000000001100000
y= 00000000000000000000000000000010
z= 00000000000000000000000001100000

4
pc= 0
x= 00000000000000000000000011000000
y= 00000000000000000000000000000001
z= 00000000000000000000000001100000

5
pc= 0
x= 00000000000000000000000011000000
y= 00000000000000000000000000000000
z= 00000000000000000000000100100000

6
pc= 1
x= 00000000000000000000000011000000
y= 00000000000000000000000000000000
z= 00000000000000000000000100100000



Provar invariante

In [6]:
def bmc_always(declare,init,trans,inv,K):
    for k in range(1,K+1):
        with Solver(name="z3") as s:

            trace = [declare(i) for i in range(k)]
    
            s.add_assertion(init(trace[0]))
            for i in range(k-1):
                s.add_assertion(trans(trace[i], trace[i+1]))
            
            # 
            s.add_assertion(Not(inv(trace[k-1])))
            if s.solve():
                for i in range(k):
                    print(i)
                    print("pc=", s.get_value(trace[i]['pc']))
                    print("x=", s.get_value(trace[i]['x']))
                return
            
    print("A propriedade é válida para o invariante")

def P(state):
    return Equals(BVAdd(BVMul(state['x'],state['y']),state['z']), BVMul(BV(a,n),BV(b,n)))

In [5]:
bmc_always(declare,init,trans,P,3)

A propriedade é válida para o invariante
